In [20]:
import mxnet as mx
import numpy as np
import sys
import os
import time
sys.path.append(os.path.dirname(os.getcwd()))
from convert_symbol import converter

In [21]:
prefix="../cnn_models/resnet20_clustered_naive_4x"
epoch=0
sym, args, auxs = mx.mod.module.load_checkpoint(prefix, epoch)
batch_size=2
data_path="../dataset/cifar10_val.rec"
val_iter=mx.image.ImageIter(batch_size=batch_size,data_shape=(3,32,32),path_imgrec=data_path)
testarray=val_iter.next().data
test_iter=mx.io.NDArrayIter(testarray,batch_size=batch_size)

In [22]:
def get_out(symbol_input,in_args,in_auxs=None):
    mod=mx.mod.Module(symbol=symbol_input, context=mx.gpu())
    mod.bind(for_training=False, data_shapes=test_iter.provide_data)
    mod.set_params(in_args, in_auxs)
    begin=time.time()
    score = mod.predict(eval_data=test_iter)
    dur = time.time() - begin
    return score, dur

In [23]:
sym_in=sym.get_internals()['stage3_unit1_relu2_output']
sym_out_original=sym.get_internals()['stage3_unit1_conv2_output']

In [24]:
tensor_in,_=get_out(sym_in, args,auxs)
tensor_out_original,_=get_out(sym_out_original, args,auxs)

In [25]:
#test1_arg ={'weight': args['stage3_unit1_conv2_weight']}
wshape=args['stage3_unit1_conv2_weight'].shape
args['test1_weight'] = args['stage3_unit1_conv2_weight']
weight_sym=mx.sym.Variable("test1_weight")
sym_test = mx.sym.Convolution(sym_in, weight=weight_sym, num_filter=wshape[0], kernel = (3,3), pad = (1,1), no_bias=True)

tensor_test, time1 = get_out(sym_test,args,auxs)


In [26]:
tensor_test.shape

(2L, 64L, 8L, 8L)

In [27]:
np.array_equal(tensor_out_original.asnumpy(),tensor_test.asnumpy())

True

In [28]:
cv = converter(prefix, epoch, batch_size=2, data_path="../dataset/cifar10_val.rec", shrink = 4)

processing symbol
symbol processed, clusters extracted.


In [29]:
def convolve_codebook_light(data, layer, name, num_filter = None, kernel=(3,3), stride=(1,1), pad=(1,1),
                                  no_bias=True):
    fshape = layer["f_shape"]
    codebookshape = layer["c_shape"]
    indices_shape = layer["i_shape"]
    output_shape = layer["out_shape"]
    filters = mx.sym.Variable(name+"_weight", shape=codebookshape)
    indices = mx.sym.Variable(name+"_indices", shape=indices_shape)
    # fshape  = codebookshape #4,16,3,3
    # filters = mx.sym.transpose(filters, axes=(1,0,2,3)).reshape((-1,1,0, 0)) #TODO: transpose is unnecessary!!
    res = mx.sym.Convolution(data=data, weight=filters, num_group=fshape[1], num_filter=fshape[0] * fshape[1], stride = stride,
                             no_bias=no_bias, kernel=kernel, pad=pad)
    res = res.reshape((0, 0, -1))  # flatten the image for matmul lookup
    res = mx.sym.batch_dot(lhs=indices, rhs=res)
    res = res.reshape((0, 0, output_shape[2], output_shape[3]))
    return res

In [30]:
#test1_arg ={'weight': args['stage3_unit1_conv2_weight']}
wshape=args['stage3_unit1_conv2_weight'].shape
args['test2_weight'] = cv.codebook_args['stage3_unit1_conv2_weight']
args['test2_indices'] =  cv.codebook_args['stage3_unit1_conv2_indices']

sym_test2 = convolve_codebook_light(sym_in, name="test2", layer = cv.network['stage3_unit1_conv2'], kernel = (3,3), pad = (1,1), no_bias=True)

tensor_test2, time2 = get_out(sym_test2,args,auxs)


In [31]:
np.mean(np.square(tensor_out_original.asnumpy()-tensor_test2.asnumpy()))

2.7678546e-13

In [32]:
print time1
print time2

0.000504016876221
0.00086498260498


In [ ]:
cv.convert()

In [ ]:
tensor_test2_2=get_out(cv.converted_sym.get_internals()['stage3_unit1_conv2_reshape_output'], cv.codebook_args, auxs)

In [ ]:
cv.converted_sym.get_internals().list_outputs()

In [ ]:
np.mean(np.square(tensor_test2.asnumpy()-tensor_test2_2.asnumpy()))

In [ ]:
sym_original_output=sym#.get_internals()['softmax_output']
tensor_original_output=get_out(sym_original_output, args,auxs)

clustered_output=get_out(cv.converted_sym, cv.codebook_args, auxs)

np.mean(np.square(tensor_original_output.asnumpy()-clustered_output.asnumpy()))

In [ ]:
    def get_score(symbol_input,in_args,in_auxs=None):
        mod=mx.mod.Module(symbol=symbol_input, context=mx.gpu())
        mod.bind(for_training=False, data_shapes=val_iter.provide_data, label_shapes=val_iter.provide_label)
        mod.set_params(in_args, in_auxs)
        return mod.score(val_iter,['acc'])

In [ ]:
import time

In [ ]:
begin=time.time()
print get_score(sym,args,auxs)
print time.time() - begin

In [ ]:
begin=time.time()
get_score(cv.converted_sym,cv.codebook_args,auxs)
print time.time() - begin